In [1]:
import os
import random
import numpy as np

import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


In [2]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# For deterministic behavior (slightly slower, but safer)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
DATASET_ROOT = r"C:\Users\ADMIN\Documents\AM Project\Pandora_18k - AM"


In [5]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


In [6]:
full_dataset = datasets.ImageFolder(
    root=DATASET_ROOT,
    transform=train_transform  # temporary; val transform applied later
)

class_names = full_dataset.classes
num_classes = len(class_names)

print("Number of classes:", num_classes)
print("Class names:", class_names)


Number of classes: 11
Class names: ['02_Early_Renaissance', '03_Northern_Renaissance', '04_High_Renaissance', '05_Baroque', '08_Realism', '09_Impressionism', '10_Post_Impressionism', '11_Expressionism', '13_Fauvism', '14_Cubism', '16_AbstractArt']


In [7]:
dataset_size = len(full_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(SEED)
)

# Important: assign correct transform to validation set
val_dataset.dataset.transform = val_transform

print(f"Total images: {dataset_size}")
print(f"Training images: {train_size}")
print(f"Validation images: {val_size}")


Total images: 11035
Training images: 8828
Validation images: 2207


In [8]:
def get_dataloaders(batch_size):
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    return train_loader, val_loader


In [9]:
from torchvision.models import (
    efficientnet_b0,
    efficientnet_b1,
    efficientnet_b2,
    efficientnet_b3,
    efficientnet_b4,
    efficientnet_b5,
    efficientnet_b6,
    efficientnet_b7
)


In [10]:
EFFICIENTNET_MODELS = {
    "b0": efficientnet_b0,
    "b1": efficientnet_b1,
    "b2": efficientnet_b2,
    "b3": efficientnet_b3,
    "b4": efficientnet_b4,
    "b5": efficientnet_b5,
    "b6": efficientnet_b6,
    "b7": efficientnet_b7,
}


In [11]:
def build_efficientnet(model_name, num_classes):
    """
    Builds a pretrained EfficientNet model with a custom classifier head.
    Backbone is frozen for baseline experiments.
    """
    if model_name not in EFFICIENTNET_MODELS:
        raise ValueError(f"Invalid EfficientNet variant: {model_name}")

    # Load pretrained model
    model = EFFICIENTNET_MODELS[model_name](weights="IMAGENET1K_V1")

    # Freeze all backbone parameters
    for param in model.parameters():
        param.requires_grad = False

    # Replace classifier head
    in_features = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(in_features, num_classes)

    # Move to device
    model = model.to(device)

    return model


In [12]:
test_model = build_efficientnet("b0", num_classes)

print(test_model.classifier)


Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=11, bias=True)
)


In [13]:
trainable_params = sum(p.requires_grad for p in test_model.parameters())
total_params = sum(1 for _ in test_model.parameters())

print(f"Trainable params: {trainable_params}")
print(f"Total params: {total_params}")


Trainable params: 2
Total params: 213


In [14]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()


In [15]:
def compute_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    return correct / labels.size(0)


In [16]:
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += (preds == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples

    return epoch_loss, epoch_acc


In [17]:
def validate_one_epoch(model, dataloader, criterion):
    model.eval()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += (preds == labels).sum().item()
            total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples

    return epoch_loss, epoch_acc


In [18]:
def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    num_epochs
):
    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(
            model, train_loader, optimizer, criterion
        )

        val_loss, val_acc = validate_one_epoch(
            model, val_loader, criterion
        )

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch [{epoch+1}/{num_epochs}] | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
        )

    return history


In [19]:
NUM_EPOCHS = 10


In [20]:
model = build_efficientnet("b0", num_classes)


In [21]:
train_loader, val_loader = get_dataloaders(batch_size=16)


In [24]:
import torch
import torch.nn as nn

# -----------------------------
# Loss
# -----------------------------
criterion = nn.CrossEntropyLoss()

# -----------------------------
# Accuracy helper
# -----------------------------
def compute_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    return correct / labels.size(0)

# -----------------------------
# Train one epoch
# -----------------------------
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += (preds == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples

    return epoch_loss, epoch_acc

# -----------------------------
# Validate one epoch
# -----------------------------
def validate_one_epoch(model, dataloader, criterion):
    model.eval()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += (preds == labels).sum().item()
            total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples

    return epoch_loss, epoch_acc

# -----------------------------
# Full training loop
# -----------------------------
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs):
    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(
            model, train_loader, optimizer, criterion
        )

        val_loss, val_acc = validate_one_epoch(
            model, val_loader, criterion
        )

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch [{epoch+1}/{num_epochs}] | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
        )

    return history

# -----------------------------
# Optimizer (BASELINE)
# -----------------------------
def get_optimizer(model, lr=1e-3):
    return torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr
    )


In [25]:
get_optimizer


<function __main__.get_optimizer(model, lr=0.001)>

In [26]:
optimizer = get_optimizer(model, lr=1e-3)


In [27]:
history = train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    num_epochs=NUM_EPOCHS
)


Epoch [1/10] | Train Loss: 1.6256, Train Acc: 0.4400 | Val Loss: 1.3772, Val Acc: 0.5220
Epoch [2/10] | Train Loss: 1.3423, Train Acc: 0.5249 | Val Loss: 1.3023, Val Acc: 0.5365
Epoch [3/10] | Train Loss: 1.2735, Train Acc: 0.5510 | Val Loss: 1.2691, Val Acc: 0.5478
Epoch [4/10] | Train Loss: 1.2283, Train Acc: 0.5632 | Val Loss: 1.2321, Val Acc: 0.5573
Epoch [5/10] | Train Loss: 1.2111, Train Acc: 0.5684 | Val Loss: 1.2430, Val Acc: 0.5541
Epoch [6/10] | Train Loss: 1.1943, Train Acc: 0.5735 | Val Loss: 1.2342, Val Acc: 0.5587
Epoch [7/10] | Train Loss: 1.1713, Train Acc: 0.5768 | Val Loss: 1.2248, Val Acc: 0.5628
Epoch [8/10] | Train Loss: 1.1601, Train Acc: 0.5880 | Val Loss: 1.2332, Val Acc: 0.5605
Epoch [9/10] | Train Loss: 1.1674, Train Acc: 0.5819 | Val Loss: 1.2391, Val Acc: 0.5618
Epoch [10/10] | Train Loss: 1.1594, Train Acc: 0.5836 | Val Loss: 1.2467, Val Acc: 0.5573


In [30]:
import os

BASE_RESULTS_DIR = r"C:\Users\ADMIN\Documents\AM Project\comparative-study-art-movement-classification\results\efficientnet_family"
os.makedirs(BASE_RESULTS_DIR, exist_ok=True)


In [31]:
BATCH_SIZE_MAP = {
    "b0": 16,
    "b1": 16,
    "b2": 16,
    "b3": 16,
    "b4": 8,
    "b5": 8,
    "b6": 4,
    "b7": 4
}


In [32]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [33]:
def plot_and_save_confusion_matrix(
    model,
    dataloader,
    class_names,
    save_path,
    normalize=False
):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    cm = confusion_matrix(all_labels, all_preds)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm,
        xticklabels=class_names,
        yticklabels=class_names,
        cmap="Blues",
        annot=False
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Normalized Confusion Matrix" if normalize else "Confusion Matrix")
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()


In [34]:
def plot_and_save_curves(history, save_dir):
    epochs = range(1, len(history["train_loss"]) + 1)

    # Loss
    plt.figure()
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["val_loss"], label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Loss Curve")
    plt.savefig(os.path.join(save_dir, "loss.png"))
    plt.close()

    # Accuracy
    plt.figure()
    plt.plot(epochs, history["train_acc"], label="Train Accuracy")
    plt.plot(epochs, history["val_acc"], label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Accuracy Curve")
    plt.savefig(os.path.join(save_dir, "accuracy.png"))
    plt.close()


In [35]:
NUM_EPOCHS = 10
LR = 1e-3

for model_name in EFFICIENTNET_MODELS.keys():
    print(f"\n========== Training EfficientNet-{model_name.upper()} ==========")

    # Create model-specific result directory
    model_dir = os.path.join(BASE_RESULTS_DIR, f"B{model_name[-1]}")
    os.makedirs(model_dir, exist_ok=True)

    # Build model
    model = build_efficientnet(model_name, num_classes)

    # DataLoaders
    batch_size = BATCH_SIZE_MAP[model_name]
    train_loader, val_loader = get_dataloaders(batch_size)

    # Optimizer
    optimizer = get_optimizer(model, lr=LR)

    # Train
    history = train_model(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        num_epochs=NUM_EPOCHS
    )

    # Save model weights
    torch.save(
        model.state_dict(),
        os.path.join(model_dir, "best_model.pth")
    )

    # Save curves
    plot_and_save_curves(history, model_dir)

    # Confusion matrices
    plot_and_save_confusion_matrix(
        model,
        val_loader,
        class_names,
        os.path.join(model_dir, "confusion_matrix.png"),
        normalize=False
    )

    plot_and_save_confusion_matrix(
        model,
        val_loader,
        class_names,
        os.path.join(model_dir, "confusion_matrix_normalized.png"),
        normalize=True
    )

    print(f"Saved results to {model_dir}")



========== Training EfficientNet-B0 ==========
Epoch [1/10] | Train Loss: 1.6216, Train Acc: 0.4414 | Val Loss: 1.3530, Val Acc: 0.5265
Epoch [2/10] | Train Loss: 1.3444, Train Acc: 0.5280 | Val Loss: 1.2692, Val Acc: 0.5460
Epoch [3/10] | Train Loss: 1.2699, Train Acc: 0.5490 | Val Loss: 1.2597, Val Acc: 0.5428
Epoch [4/10] | Train Loss: 1.2328, Train Acc: 0.5587 | Val Loss: 1.2401, Val Acc: 0.5587
Epoch [5/10] | Train Loss: 1.1944, Train Acc: 0.5757 | Val Loss: 1.2402, Val Acc: 0.5510
Epoch [6/10] | Train Loss: 1.1928, Train Acc: 0.5726 | Val Loss: 1.2431, Val Acc: 0.5523
Epoch [7/10] | Train Loss: 1.1776, Train Acc: 0.5780 | Val Loss: 1.2500, Val Acc: 0.5514
Epoch [8/10] | Train Loss: 1.1761, Train Acc: 0.5732 | Val Loss: 1.2330, Val Acc: 0.5618
Epoch [9/10] | Train Loss: 1.1593, Train Acc: 0.5853 | Val Loss: 1.2277, Val Acc: 0.5632
Epoch [10/10] | Train Loss: 1.1537, Train Acc: 0.5881 | Val Loss: 1.2423, Val Acc: 0.5555
Saved results to C:\Users\ADMIN\Documents\AM Project\comparat